# Download the file from Google Drive 

In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the xlsl file: WebCrawlerResult.xlsl
id = '1ctuL1Q4Guniyly8ZhcYddY23fUQbv4sq'
crawler_result = drive.CreateFile({'id':id}) 
crawler_result.GetContentFile('WebCrawlerResult.xlsl') 

# Data Overview

In [30]:
import pandas as pd

df = pd.read_excel('WebCrawlerResult.xlsl',header = None)
print('The total number of data is: %d' %len(df))
df.head()

The total number of data is: 1000


,0
0,很喜欢。用起来很好用，支持在薇娅那买的。
1,用两次用了爆痘，回来减轻用量不爆痘了.效果还没看到
2,过年拍的新年礼盒，结果发个普通的过来，客服沟通也不是很积极，本来想退货但是对产品还是比较认可...
3,想买一瓶精华，因为是李现代言的就买了，现在在用效果还不错，送的钥匙扣很好看，小老鼠包包也丑萌...
4,很好


# Data Preprocessing

In [54]:
import re

reviews = []
processed_reviews = []
for d in df[0]:
  # 去除 "此用户没有填写评论!" 这样无用的评论
  if(str(d) != '此用户没有填写评论!'):
    reviews.append(str(d))
    processed_reviews.append( re.split('[:：;；。./`~。...]',str(d)))

print(reviews[:5])
print("After preprocessed, the number of reviews is:", len(reviews))
print(processed_reviews[:5])

['很喜欢。用起来很好用，支持在薇娅那买的。', '用两次用了爆痘，回来减轻用量不爆痘了.效果还没看到', '过年拍的新年礼盒，结果发个普通的过来，客服沟通也不是很积极，本来想退货但是对产品还是比较认可，朋友推荐都说效果不错，使用感觉一下吧！', '想买一瓶精华，因为是李现代言的就买了，现在在用效果还不错，送的钥匙扣很好看，小老鼠包包也丑萌丑萌的，以后也会尝试用一下雅诗兰黛的其他产品。可惜没有送李现的照片，拍的太好看了。', '很好']
After preprocessed, the number of reviews is: 923
[['很喜欢', '用起来很好用，支持在薇娅那买的', ''], ['用两次用了爆痘，回来减轻用量不爆痘了', '效果还没看到'], ['过年拍的新年礼盒，结果发个普通的过来，客服沟通也不是很积极，本来想退货但是对产品还是比较认可，朋友推荐都说效果不错，使用感觉一下吧！'], ['想买一瓶精华，因为是李现代言的就买了，现在在用效果还不错，送的钥匙扣很好看，小老鼠包包也丑萌丑萌的，以后也会尝试用一下雅诗兰黛的其他产品', '可惜没有送李现的照片，拍的太好看了', ''], ['很好']]


# Analyzed Data

In [0]:
import numpy as np

def judge_comment(de,reviews):
  judges = pd.DataFrame(np.zeros(11*len(df)).reshape(len(df),11),
                        columns = ['品牌','物流正面','物流负面','包装正面','包装负面','效果正面','效果负面','赠品正面','赠品负面','性价比正面','性价比负面'])
  
  for i in range(len(reviews)):
    words = reviews[i]
    for word in words:
      # 品牌相关
      if '雅诗兰黛' in word:
        judeges.iloc[i]['品牌'] = 1

      # 物流相关
      if '物流' in word or '快递' in word or '配送' in word or '送货' in word or '速度'in word:
        if '好' in word or '不错' in word or '棒' in word or '满意' in word or '速度' in word or '迅速' in word:
          judges.iloc[i]['物流正面'] = 1
        if '慢' in word or '龟速' in word or '暴力' in word or '不行' in word or '等' in word or '很久' in word:
          judges.iloc[i]['物流负面'] = 1
      
      # 包装相关
      if  '盒子' in word or '包装' in word or '袋子' in word or '外观' in word or '瓶子' in word or '礼盒' in word:
        if '高端' in word or '大气' in word or '上档次' in word or '严实' in word or '精美' in word or '气派' in word or '精致' in word or '完美' in word or '漂亮' in word:
          judges.iloc[i]['包装正面'] = 1
        if '简陋' in word or '破' in word or '寒酸' in word or '坏' in word or '漏' in word or '土' in word or '丑' in word:
          judges.iloc[i]['包装负面'] = 1
      
      # 效果相关
      if '效果' in word or '保湿' in word or '控油' in word or '修复' in word or '吸收' in word or '补水' in word:
        if '有效' in word or '很棒' in word or '很快' in word or '好' in word or '厉害' in word or '不错' in word or '惊喜' in word:
          judges.iloc[i]['效果正面'] = 1
        if '没用' in word or  '没' in word or '不好用' in word or '油腻' in word or '差' in word or '不行' in word or '致痘' in word:
          judges.iloc[i]['效果负面'] = 1
    
      # 赠品相关
      if '小样' in word or  '赠品' in word:
        if '很多' in word or '一堆' in word or '给力' in word or '不少' in word or '喜欢' in word:
          judges.iloc[i]['赠品正面'] = 1
        if '很少' in word or '不好' in word or '坏掉' in word or '不喜欢' in word:
          judges.iloc[i]['赠品负面'] = 1

      # 性价比相关
      if '划算' in word or '便宜' in word or '赚了' in word or '超值' in word or '囤货' in word or '性价比高' in word or '不贵' in word or '物美价廉' in word:
        judges.iloc[i]['性价比正面']  = 1
      elif '贵' in word or '不值' in word or '亏了' in word or '不划算' in word or '不便宜' in word or '性价比低' in word:
        judges.iloc[i]['性价比负面']  = 1    
 
  final_result = pd.concat([df,judges],axis = 1)
  return final_result

In [61]:
judges = judge_comment(df,reviews)
judges.head(20)

,0,品牌,物流正面,物流负面,包装正面,包装负面,效果正面,效果负面,赠品正面,赠品负面,性价比正面,性价比负面
0,很喜欢。用起来很好用，支持在薇娅那买的。,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,用两次用了爆痘，回来减轻用量不爆痘了.效果还没看到,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,过年拍的新年礼盒，结果发个普通的过来，客服沟通也不是很积极，本来想退货但是对产品还是比较认可...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,想买一瓶精华，因为是李现代言的就买了，现在在用效果还不错，送的钥匙扣很好看，小老鼠包包也丑萌...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,很好,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,此用户没有填写评论!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,还没用,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,此用户没有填写评论!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,棒棒哒,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,此用户没有填写评论!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
rank = judges.iloc[:,2:].sum().reset_index().sort_values(0,ascending = False)
rank.columns = ['分类','提及次数']
rank['占比'] = rank['提及次数'] / rank['提及次数'].sum()
rank['高级分类'] = rank['分类'].str[:-2]
rank

,分类,提及次数,占比,高级分类
9,性价比负面,22.0,1.0,性价比
0,物流正面,0.0,0.0,物流
1,物流负面,0.0,0.0,物流
2,包装正面,0.0,0.0,包装
3,包装负面,0.0,0.0,包装
4,效果正面,0.0,0.0,效果
5,效果负面,0.0,0.0,效果
6,赠品正面,0.0,0.0,赠品
7,赠品负面,0.0,0.0,赠品
8,性价比正面,0.0,0.0,性价比
